In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.9.0


### Generate Torque Training Data and Labels 

In [132]:
# Parameter space restricting force vector lengths in y direction and defining length of door along x direction
x_min = 0
x_max = 10
y_min = -5
y_max = 5

def random_examples(N):
    """ Generate random force vector pairs
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_min
    data[:,6] *= x_max
    data[:,7] *= 0
    
    return data

def perp_only_close(N):
    """ Generate random force vector pairs perpendicular to lever arm that result in 0 net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N,8)
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] = data[:,0]
    data[:,3] *= 0
    
    data[:,4] *= x_max
    data[:,5] = - data[:,0] * data[:,1] / data[:,4]
    data[:,6] = data[:,4]
    data[:,7] *= 0
    
    return data
    
def perp_only_open(N):
    """ Generate random force vector pairs perpendicular to lever arm that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= data[:,0]
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_min
    data[:,6] = data[:,4]
    data[:,7] *= 0
    
    return data

def same_dist_close(N): 
    """ Generate random force vector pairs terminating at same point on lever arm that result in 0 net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,6] = data[:,2]
    data[:,5] = - data[:,2] * data[:,1] / data[:,6]
    data[:,7] *= 0
    
    return data

def same_dist_open(N):
    """ Generate random force vector pairs terminating at same point on lever arm that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_max
    data[:,6] = data[:,2]
    data[:,7] *= 0
    
    return data
    
def same_mag_close(N): 
    """ Generate random force vector pairs with same magnitude that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    pass

def same_mag_open(N):
    """ Generate random force vector pairs with same magnitude that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    pass

N_rand = 10000
N_perp_open = 10000
N_perp_close = 10000
N_dist_close = 10000
N_dist_open = 10000
N_mag_close = 10
N_mag_open = 10

N = N_rand + N_perp_open + N_perp_close + N_dist_close + N_dist_open + N_mag_close + N_mag_open

#TODO: Finish populating with data
data = np.vstack((random_examples(N_rand), perp_only_close(N_perp_close), perp_only_open(N_perp_open),
                    same_dist_close(N_dist_close), same_dist_open(N_dist_open))) 

labels = np.array([i[1] * i[2] + i[5] * i[6] == 0 for i in data]).astype(int)

N_test = 100
test_data = np.vstack((random_examples(N_test), perp_only_close(N_test), perp_only_open(N_test),
                    same_dist_close(N_test), same_dist_open(N_test))) 
test_labels = np.array([i[1] * i[2] + i[5] * i[6] == 0 for i in test_data]).astype(int)


### Preprocessing
- TODO: Normalize data for input into NN.

In [70]:
print(1 in labels)

True


### Create Model 
- TODO: Create model using NN architecture from tensorflow

In [71]:
print(data.shape)
print(labels.shape)

(250, 8)
(250,)


In [118]:
#TODO: Figure out optimal network architecture for this class of problem

model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dense(80, activation=tf.nn.relu),
    keras.layers.Dense(60, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [133]:
model.fit(data, labels, epochs=5)

Epoch 1/5
50000/50000 [==============================] - 3s 52us/step - loss: 0.1296 - acc: 0.9585
Epoch 2/5
50000/50000 [==============================] - 3s 50us/step - loss: 0.0982 - acc: 0.9693
Epoch 3/5
50000/50000 [==============================] - 2s 49us/step - loss: 0.0927 - acc: 0.9716
Epoch 4/5
50000/50000 [==============================] - 2s 50us/step - loss: 0.0898 - acc: 0.9718
Epoch 5/5
50000/50000 [==============================] - 3s 50us/step - loss: 0.0867 - acc: 0.9735


### Test Model accuracy on new data

In [134]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print('Test accuracy:', test_acc)

500/500 [==============================] - 0s 24us/step
Test accuracy: 0.974


### Small Data Model 
- Low complexity (high bias) model for small data
- Same number of layers as features at least (in this case, 8). 